### Sunday, December 3, 2023

Downloaded the "TheBloke/Python-Code-33B-GPTQ" model yesterday ... ran a quick test this morning. It works!

### Friday, November 17, 2023

https://huggingface.co/TheBloke/Python-Code-33B-GPTQ

In [1]:
# You need this to just target the 4090.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model_name_or_path = "TheBloke/Python-Code-33B-GPTQ"

In [4]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt
models--TheBloke--Python-Code-33B-GPTQ


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [6]:
%%time
# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")


# Load time
# CPU times: user 19.9 s, sys: 9.32 s, total: 29.3 s
# Wall time: 12.4 s
# 18424MiB VRAM

# Download time
# CPU times: user 1min 32s, sys: 1min 39s, total: 3min 12s
# Wall time: 4h 37s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 19.9 s, sys: 9.32 s, total: 29.3 s
Wall time: 12.4 s


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--Python-Code-33B-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 16.9GB to /home/rob/Data3/huggingface/transformers

In [7]:
prompt = "Tell me about AI"
prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

# print("\n\n*** Generate:")

In [8]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

In [9]:
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# 19524 MiB VRAM

# 9.5s
# 14.9s

<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:

Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program that can perceive its environment, make decisions, and take actions to achieve a goal. 

AI researchers study various aspects of intelligent agents, such as their learning algorithms, decision-making processes, and communication skills. They also explore how AI can be applied in different fields, such as computer games, finance, and healthcare.

One area of AI that has received a lot of attention recently is deep learning. Deep learning is a subfield of AI that studies neural networks, which are inspired by the structure of the brain. Deep learning algorithms have achieved impressive results in areas such as image recognition and language processing.

Overa

In [10]:
# Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [11]:
print(pipe(prompt_template)[0]['generated_text'])

# 19906 MiB VRAM

# 12.9s 
# 15.2s

This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
Artificial Intelligence (AI) is a branch of computer science that studies intelligent agents. An intelligent agent is a system that perceives its environment and takes actions to achieve its goals. 

In the field of AI, we study how to build such intelligent agents using machines. We use various techniques such as machine learning, data mining, and logical reasoning to create algorithms that can solve complex problems.

Some examples of AI applications include:

1. Self-driving cars: AI systems can process large amounts of data from sensors and make decisions about how to move the car.

2. Robotics: AI systems can be used to control robots, allowing them to perform tasks such as manipulating objects or navigating through obstacles.

3. Drones: AI systems can help drones navigate through